In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [3]:
import yfinance as yf
xmr = yf.Ticker("XMR-USD")
df_xmr = yf.download(tickers = "XMR-USD",
                     start = "2022-09-07",
                     end = "2023-09-09" ,
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score , recall_score , precision_score , confusion_matrix, auc
from sklearn.model_selection import GridSearchCV , train_test_split

In [11]:
n_days = 14
## values of close column
close = df_xmr["Close"]
# array to save input and output for each day
X_s = []
y_s = []
# arrays to save scores for train data
f_train = []
r_train = []
p_train = []
# array to save scores for validation data
f_val = []
r_val = []
p_val = []
# calc scores for each day
for i in range( 1, n_days) :
    X = []
    y = []
    # make input and output datas for each day
    for j in range(i,len(close)) :
        f = []
        for l in range(i) :
            f.append(close[j-l])
        X.append(f)
        y.append((close[j]-close[j-1])> 0)
    X_s.append(X)
    y_s.append(y)
    #params = {'loss': ['log_loss', 'exponential'] , 'learning_rate' : [0.1 , 0.5 , 1] , 'n_estimators' : [100 , 500 , 1000] , 'max_depth' : [3,5,8,None]}
    gnb = GaussianNB()
    print("n day : ",i)
    # make train and validation data
    x_train , x_val , y_train , y_val = train_test_split(X , y , test_size = 0.3 , random_state = 42)
    gnb.fit(x_train , y_train)
    # predict using train and val data
    y_pred_train = gnb.predict(x_train)
    y_pred_val = gnb.predict(x_val)
    # calc scores for train data and append them
    f1_train = f1_score(y_train , y_pred_train)
    recall_train = recall_score(y_train,y_pred_train)
    precision_train = precision_score(y_train,y_pred_train)
    f_train.append(f1_train)
    r_train.append(recall_train)
    p_train.append(precision_train)
    # calc scores for validation data and append them
    f1_val = f1_score(y_val , y_pred_val)
    recall_val = recall_score(y_val , y_pred_val)
    precision_val = precision_score(y_val , y_pred_val)
    f_val.append(f1_val)
    r_val.append(recall_val)
    p_val.append(precision_val)

n day :  1
n day :  2
n day :  3
n day :  4
n day :  5
n day :  6
n day :  7
n day :  8
n day :  9
n day :  10
n day :  11
n day :  12
n day :  13


In [18]:
for i in range( n_days-1) : 
    print(f"day {i+1}")
    print(f"for train data : \nf1 score :{f_train[i]}\nrecall : {r_train[i]}\nprecision : {p_train[i]}" )
    print(f"for validation data : \nf1 score :{f_val[i]}\nrecall : {r_val[i]}\nprecision : {p_val[i]}" )
    print("*************************************************************")

day 1
for train data : 
f1 score :0.5813148788927336
recall : 0.6131386861313869
precision : 0.5526315789473685
for validation data : 
f1 score :0.5737704918032787
recall : 0.5384615384615384
precision : 0.6140350877192983
*************************************************************
day 2
for train data : 
f1 score :0.7964071856287424
recall : 0.9779411764705882
precision : 0.6717171717171717
for validation data : 
f1 score :0.8311688311688311
recall : 0.9846153846153847
precision : 0.7191011235955056
*************************************************************
day 3
for train data : 
f1 score :0.7349081364829396
recall : 0.9655172413793104
precision : 0.5932203389830508
for validation data : 
f1 score :0.6751592356687898
recall : 0.9636363636363636
precision : 0.5196078431372549
*************************************************************
day 4
for train data : 
f1 score :0.7667731629392971
recall : 0.8571428571428571
precision : 0.6936416184971098
for validation data : 
f1 score :

In [25]:
# test data frame
df_test = yf.download(tickers = "XMR-USD",
                     start = "2023-09-04",
                     end = "2023-10-07" ,
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [32]:
best_day = 2
close_test = df_test["Close"]
x_test = []
y_test = []
close = df_xmr["Close"]
X = []
y = []
# make input and output datas for best day
for j in range(best_day,len(close)) :
    f = []
    for l in range(best_day) :
        f.append(close[j-l])
    X.append(f)
    y.append((close[j]-close[j-1])> 0)
    
for i in range(best_day,len(close_test)) : 
    f = []
    for j in range(best_day) : 
        f.append(close_test[i-j])
    x_test.append(f)
    y_test.append(close_test[i]-close_test[i-1] > 0 )
gnb = GaussianNB()
gnb.fit(X,y)
y_pred_test = gnb.predict(x_test)
f_test = f1_score(y_test , y_pred_test)
r_test = recall_score(y_test , y_pred_test)
p_test = precision_score(y_test , y_pred_test)
print(f"for test data : \nf1 score : {f_test} , recall score : {r_test} , precision score : {p_test}")

for test data : 
f1 score : 0.76 , recall score : 1.0 , precision score : 0.6129032258064516
